## 📊 AEMR Energy Outage Analysis

**Project Type:** SQL Data Analysis  
**Tools Used:** SQLite, SQLAlchemy, Jupyter Notebook  
**Objective:** Analyze outage event data for the American Energy Market Regulator (AEMR) to uncover patterns in energy disruptions and identify potential reliability issues among providers.

### Key Questions Explored:
- Which types of outages are most common and how long do they last?
- What trends exist across 2016 and 2017?
- Are certain providers less reliable than others?

---

### Data Loading

In [1]:
%%capture
!pip install ipython-sql sqlalchemy
import sqlalchemy
sqlalchemy.create_engine("sqlite:///AEMR.db")
%load_ext sql
%sql sqlite:///AEMR.db

In [2]:
%%js
require(['notebook/js/codecell'], function (codecell) {
    codecell.CodeCell.options_default.highlight_modes['magic_text/x-mssql'] = { 'reg': [/%?%sql/] };
    Jupyter.notebook.events.one('kernel_ready.Kernel', function () {
        Jupyter.notebook.get_cells().map(function (cell) {
            if (cell.cell_type == 'code') { cell.auto_highlight(); }
        });
    });
});

<IPython.core.display.Javascript object>

In [3]:
%%sql
SELECT COUNT(*) AS total_rows
FROM AEMR_Outage_Table;

 * sqlite:///AEMR.db
Done.


total_rows
4655


In [4]:
%%sql
SELECT *
FROM AEMR_Outage_Table
LIMIT 3;

 * sqlite:///AEMR.db
Done.


EventID,Start_Time,End_Time,Year,Month,Facility_Code,Participant_Code,Status,Outage_Reason,Energy_Lost_MW,Description_Of_Outage
1,2017-12-28 06:00,2017-12-28 10:00,2017,12,DNHR_DENMARK_WF1,DNHR,Approved,Consequential,1.44,a network outage in Denmark caused the windfarm to trip at 6:26 am
2,2017-12-26 09:00,2017-12-27 00:00,2017,12,COLLGAR_WF1,COLLGAR,Approved,Forced,30.0,Forced Outage - BS on Overhead line pole S-81 - CG10/11
3,2017-12-31 09:00,2017-12-31 09:00,2017,12,AURICON_PNJ_U1,AURICON,Approved,Forced,5.702999999999999,Under generation - ambient conditions


### Part I. Energy Stability & Market Outages 

####  What are the most common outage types? 

To begin the analysis, we want to see the total number of approved outages across both 2016 and 2017. This includes all outage types — Forced, Consequential, Scheduled, and Opportunistic. 

In [8]:
%%sql
SELECT count(Status) as Total_Number_Outages, Outage_Reason,  Year
FROM AEMR_Outage_Table
WHERE Status = "Approved"
GROUP BY Outage_Reason, Year
ORDER BY Year;

 * sqlite:///AEMR.db
Done.


Total_Number_Outages,Outage_Reason,Year
181,Consequential,2016
1264,Forced,2016
106,Opportunistic Maintenance (Planned),2016
380,Scheduled (Planned),2016
127,Consequential,2017
1622,Forced,2017
102,Opportunistic Maintenance (Planned),2017
320,Scheduled (Planned),2017


We'll group the data by Year and Month and sort the results by Year, Month, and Total_Number_Outages in descending order. This helps us identify months with unusually high outage activity.

In [81]:
%%sql
SELECT Year, Month ,count(*) as Total_Number_Outages
FROM AEMR_Outage_Table
WHERE Status = "Approved"
GROUP BY Year, Month
ORDER BY Total_Number_Outages DESC, Month, Year
LIMIT 10;

 * sqlite:///AEMR.db
Done.


Year,Month,Total_Number_Outages
2017,10,276
2017,12,248
2016,2,227
2017,7,210
2017,8,209
2017,6,204
2016,11,200
2017,11,197
2016,8,194
2016,1,191


Next, we refine the analysis by also grouping the results by Outage Type, in addition to Year and Month.
This allows us to track how each outage type behaves over time and identify whether certain types are becoming more frequent in specific months when comparing 2016 to 2017.

In [87]:
%%sql
SELECT Year, Month, Outage_Reason,count(*) as Total_Number_Outages
FROM AEMR_Outage_Table
WHERE Status = "Approved"
GROUP BY Outage_Reason, Year, Month
ORDER BY Outage_Reason, Month, Year, Total_Number_Outages;

 * sqlite:///AEMR.db
Done.


Year,Month,Outage_Reason,Total_Number_Outages
2016,1,Consequential,24
2017,1,Consequential,12
2016,2,Consequential,23
2017,2,Consequential,27
2016,3,Consequential,7
2017,3,Consequential,19
2017,4,Consequential,12
2016,5,Consequential,36
2017,5,Consequential,5
2016,6,Consequential,12


**🧠 Interpretation:** 

Forced outages occur more frequently than other outage types and occur more often during June-Aug and Oct-Dec. However, their longer durations pose a far greater threat to energy reliability. To pinpoint the most critical risks, we should drill down into which providers’ forced outages last the longest, since these prolonged interruptions have the biggest impact on the grid.

In [18]:
%%sql
SELECT Participant_Code, Outage_Reason, Year,
ROUND(AVG((ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time)))),2) as Average_Duration_In_Days, 
COUNT(*) as Total_Number_Outage_Events
FROM AEMR_Outage_Table
WHERE Status = "Approved"
GROUP BY Participant_Code, Outage_Reason, Year
ORDER BY Total_Number_Outage_Events DESC, Outage_Reason, Year
LIMIT 10;

 * sqlite:///AEMR.db
Done.


Participant_Code,Outage_Reason,Year,Average_Duration_In_Days,Total_Number_Outage_Events
AURICON,Forced,2017,0.07,490
GW,Forced,2016,0.38,317
GW,Forced,2017,1.06,227
AURICON,Forced,2016,0.07,208
AUXC,Forced,2016,0.08,206
MELK,Forced,2017,2.28,177
TRMOS,Forced,2017,0.42,172
MELK,Forced,2016,0.83,157
PUG,Forced,2017,0.25,135
AUXC,Forced,2017,0.02,120


**🧠 Interpretation:** 

The participants with the highest number of outages were AURICON, GW, and AUXC. Among these, MELK and GW recorded the longest average outage durations, indicating that while frequency is high for some, duration-related impact is more severe for others.

Based from the above information, we want to classify our participants accordingly based on reliability metrics of uptime.

We classify a participant based off the following criteria:

-  High Risk - On average, the participant is unavailable for > 24 Hours (1 Day)
-  Medium Risk - On average, the participant is unavailable between 12 and 24 Hours 
-  Low Risk - On average, the participant is unavailable for less than 12 Hours 


In [21]:
%%sql

WITH Average_Duration_Time AS (
    SELECT Participant_Code, Outage_Reason,Year,
    COUNT(*) as Total_Number_Outage_Events,
    ROUND(AVG((ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time)))),2) as Average_Duration_In_Days 
    FROM AEMR_Outage_Table
    WHERE Status = "Approved"
    GROUP BY Participant_Code, Outage_Reason, Year)

SELECT Participant_Code, Outage_Reason, Year,Total_Number_Outage_Events, Average_Duration_In_Days,
    CASE
        WHEN Average_Duration_In_Days > 1 THEN 'High Risk'
        WHEN Average_Duration_In_Days > 0.5 THEN 'Medium Risk'
        ELSE 'Low Risk'
    END AS Risk_Classification
FROM Average_Duration_Time
ORDER BY Risk_Classification, Total_Number_Outage_Events DESC, Average_Duration_In_Days DESC, Year
LIMIT 10;

 * sqlite:///AEMR.db
Done.


Participant_Code,Outage_Reason,Year,Total_Number_Outage_Events,Average_Duration_In_Days,Risk_Classification
GW,Forced,2017,227,1.06,High Risk
MELK,Forced,2017,177,2.28,High Risk
MELK,Scheduled (Planned),2016,85,4.61,High Risk
PJRH,Forced,2016,81,1.22,High Risk
KORL,Forced,2017,76,1.21,High Risk
MELK,Scheduled (Planned),2017,70,6.89,High Risk
AURICON,Scheduled (Planned),2016,46,1.89,High Risk
GW,Scheduled (Planned),2016,45,4.43,High Risk
AURICON,Scheduled (Planned),2017,45,1.45,High Risk
COLLGAR,Forced,2017,45,1.38,High Risk


**🧠 Interpretation:** 

This analysis shows that GW, MELK, and PJRH have the longest average outage durations. However, to better assess true operational risk, we can dig a little deeper by refining our risk classification.

Rather than including all outage types, it makes more sense to focus solely on Forced Outages, since these are unplanned and pose the greatest threat to grid reliability. By narrowing our criteria and incorporating both average outage duration and total number of events, we define risk levels more accurately:

- High Risk – Average outage duration > 24 hours OR total events > 20
- Medium Risk – Average outage duration between 12–24 hours OR total events between 10–20
- Low Risk – Average outage duration < 12 hours OR total events < 10
- If the outage type is not Forced, the participant is marked as N/A

This more focused approach allows us to identify participants who present a meaningful risk due to unplanned and prolonged outages.

In [23]:
%%sql

WITH Average_Duration_Time AS (
    SELECT 
    Participant_Code, 
    Outage_Reason,
    Year,
    ROUND(AVG((ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time)))),2) AS Average_Duration_Day,
    COUNT(*) as Total_Number_Outage_Events
    FROM AEMR_Outage_Table
    WHERE Status = 'Approved'
    GROUP BY Participant_Code, Outage_Reason, Year)

SELECT Participant_Code, Outage_Reason, Year, Average_Duration_Day, Total_Number_Outage_Events,
    CASE
        WHEN Outage_Reason IS NOT 'Forced' THEN 'N/A'
        WHEN Average_Duration_Day > 1 OR Total_Number_Outage_Events > 20 THEN 'High Risk'
        WHEN Average_Duration_Day BETWEEN 0.5 AND 1 OR Total_Number_Outage_Events BETWEEN 10 AND 20 THEN 'Medium Risk'
        WHEN Average_Duration_Day < 0.5 OR Total_Number_Outage_Events < 10  THEN 'Medium Risk'
    END AS Risk_Classification
FROM Average_Duration_Time
ORDER BY Risk_Classification, Total_Number_Outage_Events DESC, Average_Duration_Day DESC
LIMIT 25;

 * sqlite:///AEMR.db
Done.


Participant_Code,Outage_Reason,Year,Average_Duration_Day,Total_Number_Outage_Events,Risk_Classification
AURICON,Forced,2017,0.07,490,High Risk
GW,Forced,2016,0.38,317,High Risk
GW,Forced,2017,1.06,227,High Risk
AURICON,Forced,2016,0.07,208,High Risk
AUXC,Forced,2016,0.08,206,High Risk
MELK,Forced,2017,2.28,177,High Risk
TRMOS,Forced,2017,0.42,172,High Risk
MELK,Forced,2016,0.83,157,High Risk
PUG,Forced,2017,0.25,135,High Risk
AUXC,Forced,2017,0.02,120,High Risk


**🧠 Interpretation:** 

Participants WGUTD, MELK, and ENRG recorded the longest average outage durations, indicating prolonged unavailability during their outages. On the other hand, AURICON, GW, and AUX experienced the highest number of outages, pointing to frequent disruptions in their operations.

Therefore, while some participants pose a risk due to the length of individual outages, others pose a risk due to their frequency. Both dimensions are critical when evaluating overall operational risk and should be considered in classification and mitigation strategies.

###  Part II. Energy Losses & Market Reliability

To further assess market reliability, we shift our focus to understanding the proportion of Forced Outages relative to all outage types. Since Forced Outages are unplanned and can severely impact grid stability, analyzing their share will provide valuable insight into the overall resilience of the energy market.

In [27]:
%%sql

WITH forced_outages AS (
    SELECT count(*) as forced_outages,Year
    FROM AEMR_Outage_Table
    WHERE Outage_Reason = 'Forced' AND Status = 'Approved'
    GROUP BY Year
),
total_outages AS (
    SELECT COUNT(*) as total_outages, Year
    FROM AEMR_Outage_Table
    WHERE Status = 'Approved'
    GROUP BY Year
)

SELECT Year, ROUND((forced_outages*1.0 /total_outages)*100,2) as proportion_forced_outages, forced_outages, total_outages
FROM forced_outages
JOIN total_outages
USING(Year)
GROUP BY YEAR
ORDER BY YEAR;

 * sqlite:///AEMR.db
Done.


Year,proportion_forced_outages,forced_outages,total_outages
2016,65.46,1264,1931
2017,74.71,1622,2171


**🧠 Interpretation:**

As seen in the chart, the percentage of Forced Outages increased from 65% in 2016 to 74% in 2017, indicating a rising trend in unplanned disruptions. This suggests a potential decline in operational reliability and highlights the need for closer monitoring and preventive measures.

Now that we've identified Forced Outages as a key area of concern—being unplanned and financially impactful—it's important to understand the scale of energy lost due to these disruptions.

While we’ve seen that some energy participants experience a high number of forced outages, the total outage duration alone doesn’t provide the full picture. A few major outages can skew the total, masking whether outages are frequent, prolonged, or both.

To get a clearer understanding, we'll break the analysis into two perspectives:

Macro View: Total duration of outages — gives us the overall time a participant was offline.
Micro View: Average outage duration — helps us understand how severe the outages are on a per-incident basis across the year.
Let’s explore both to better quantify the impact of energy lost and pinpoint which participants pose the greatest operational risk.

In [32]:
%%sql

Select 
    COUNT(*) as Total_Number_Outages,
    ROUND(SUM(ABS((JULIANDAY(End_Time) - JULIANDAY(Start_Time)))),2) AS Total_Duration_In_Days,
    ROUND(SUM(Energy_Lost_MW),2) AS Total_Energy_Lost,
    Outage_Reason,
    Participant_Code,
    Facility_Code,
    Year
FROM AEMR_Outage_Table
WHERE Status = 'Approved'
GROUP BY Participant_Code, Facility_Code, Outage_Reason
ORDER BY Total_Energy_Lost DESC, Year ASC
LIMIT 5;
    

 * sqlite:///AEMR.db
Done.


Total_Number_Outages,Total_Duration_In_Days,Total_Energy_Lost,Outage_Reason,Participant_Code,Facility_Code,Year
544,361.29,35077.94,Forced,GW,BW1_GREENWATERS_G2,2017
698,48.71,32335.83,Forced,AURICON,AURICON_PNJ_U1,2017
334,533.75,24056.47,Forced,MELK,MELK_G7,2017
155,874.83,17168.07,Scheduled (Planned),MELK,MELK_G7,2017
109,35.62,14741.52,Forced,PMC,PMC_AG,2017


**🧠 Interpretation:** 

The table below shows that GW, AURICON, and MELK experienced the highest total energy loss due to Forced Outages. This suggests that these participants had either prolonged unplanned outages, a high frequency of disruptions, or a combination of both, contributing significantly to overall grid instability.

In [95]:
%%sql

SELECT
    ROUND(AVG(ABS((JULIANDAY(End_Time) - JULIANDAY(Start_Time)))),2) AS Average_Duration_In_Days,
    ROUND(AVG(Energy_Lost_MW),2) AS Average_Energy_Lost,
    Outage_Reason,
    Participant_Code,
    Facility_Code,
    Year
FROM AEMR_Outage_Table
WHERE Status = 'Approved' AND Outage_Reason = 'Forced'
GROUP BY Participant_Code, Facility_Code, Year
ORDER BY Average_Energy_Lost DESC, Year ASC
LIMIT 10;

 * sqlite:///AEMR.db
Done.


Average_Duration_In_Days,Average_Energy_Lost,Outage_Reason,Participant_Code,Facility_Code,Year
1.11,149.0,Forced,COLLGAR,COLLGAR_WF1,2016
0.04,141.21,Forced,PMC,PMC_AG,2017
0.49,131.78,Forced,PMC,PMC_AG,2016
0.83,87.71,Forced,MELK,MELK_G7,2016
1.06,85.14,Forced,GW,BW1_GREENWATERS_G2,2017
0.25,80.0,Forced,TSLA_MGT,TESLA_PICTON_G1,2016
0.38,76.23,Forced,KORL,KORL_GT3,2016
1.22,72.61,Forced,PJRH,PJRH_GT11,2016
0.84,67.21,Forced,PJRH,PJRH_GT11,2017
1.38,61.93,Forced,COLLGAR,COLLGAR_WF1,2017


In [42]:
%%sql
WITH forced_energy_loss AS (
SELECT
    ROUND(AVG(Energy_Lost_MW),2) AS Average_Energy_Lost,
    ROUND(SUM(Energy_Lost_MW),2) AS Total_Energy_Lost,
    Outage_Reason,
    Participant_Code,
    Facility_Code,
    Year
FROM AEMR_Outage_Table
WHERE Status = 'Approved' AND Outage_Reason = 'Forced'
GROUP BY Outage_Reason, Participant_Code, Facility_Code, Year)

SELECT 
Average_Energy_Lost, 
Total_Energy_Lost,
ROUND((Total_Energy_Lost*1.0/ (SELECT SUM(Total_Energy_Lost) FROM forced_energy_loss) * 100),2) as Pct_Energy_Loss,
Outage_Reason,
Participant_Code,
forced_energy_loss.Facility_Code,
Year
FROM forced_energy_loss
ORDER BY Total_Energy_Lost DESC
Limit 5;

 * sqlite:///AEMR.db
Done.


Average_Energy_Lost,Total_Energy_Lost,Pct_Energy_Loss,Outage_Reason,Participant_Code,Facility_Code,Year
44.16,21639.55,14.21,Forced,AURICON,AURICON_PNJ_U1,2017
85.14,19326.56,12.69,Forced,GW,BW1_GREENWATERS_G2,2017
49.69,15751.38,10.34,Forced,GW,BW1_GREENWATERS_G2,2016
87.71,13771.07,9.04,Forced,MELK,MELK_G7,2016
51.42,10696.28,7.02,Forced,AURICON,AURICON_PNJ_U1,2016


In [47]:
%%sql
WITH forced_energy_loss AS (
    SELECT
        Participant_Code,
        Facility_Code,
        Description_Of_Outage,
        ROUND(SUM(Energy_Lost_MW),2) AS Total_Energy_Lost
    FROM AEMR_Outage_Table
    WHERE Status = 'Approved' 
      AND Outage_Reason = 'Forced'
      AND Participant_Code IN ('GW', 'MELK', 'AURICON')
    GROUP BY 
        Participant_Code, 
        Facility_Code, 
        Description_Of_Outage
),
ranked_loss AS (
    SELECT 
        Participant_Code,
        Facility_Code,
        Description_Of_Outage,
        Total_Energy_Lost,
        ROUND((Total_Energy_Lost * 1.0 / (SELECT SUM(Total_Energy_Lost) FROM forced_energy_loss)) * 100, 2) AS Pct_Energy_Loss,
        RANK() OVER (PARTITION BY Participant_Code, Facility_Code ORDER BY Total_Energy_Lost DESC) AS rank
    FROM forced_energy_loss
)

SELECT 
    Participant_Code,
    Facility_Code,
    Description_Of_Outage,
    Total_Energy_Lost,
    Pct_Energy_Loss,
    rank
FROM ranked_loss
WHERE rank = 1
ORDER BY Total_Energy_Lost DESC
LIMIT 5; 

 * sqlite:///AEMR.db
Done.


Participant_Code,Facility_Code,Description_Of_Outage,Total_Energy_Lost,Pct_Energy_Loss,rank
GW,BW1_GREENWATERS_G2,Operational Issues caused real time forced outage.,28687.54,31.36,1
AURICON,AURICON_PNJ_U1,Full unit trip,6033.87,6.6,1
MELK,MELK_G7,Safety Issues,1100.0,1.2,1


**🧠 Interpretation:** 

Having identified GW, MELK, and Auricon as the top contributors to total energy loss during forced outages, we examined the primary cause of their outages to uncover more targeted insights:

- For each participant, we identified the most common outage description responsible for the largest energy loss.
- We calculated the total energy lost (MW) and its percentage contribution to their overall energy loss footprint.
- This analysis helps pinpoint the leading operational vulnerabilities per participant, offering clear focus areas for mitigation.

## 🧾 Summary & Recommendations

- **Most Frequent Outage Type:** Forced outages were the most common across both years.
- **Seasonal Trends:** Jun-Aug and Oct-Nov experienced spikes in forced outages.
- **Reliability Issues:** GW, AURICON, and MELK experienced significantly more outages than others.

### 📌 Recommendations:
- Investigate operational practices of providers with high outage rates.
- Explore preventative maintenance strategies to reduce forced outages.
- Prioritize investments in grid stability for high-outage regions.

---